In [227]:
import numpy as np 
import pandas as pd

In [228]:
import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint

np.random.seed(500)

In [229]:
df=pd.read_csv('/content/drive/MyDrive/Deep Learning Class/watson_healthcare_modified.csv')
df.head()

,EmployeeID,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,...,RelationshipSatisfaction,StandardHours,Shift,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,1313919,41,No,Travel_Rarely,1102,Cardiology,1,2,Life Sciences,1,...,1,80,0,8,0,1,6,4,0,5
1,1200302,49,No,Travel_Frequently,279,Maternity,8,1,Life Sciences,1,...,4,80,1,10,3,3,10,7,1,7
2,1060315,37,Yes,Travel_Rarely,1373,Maternity,2,2,Other,1,...,2,80,0,7,3,3,0,0,0,0
3,1272912,33,No,Travel_Frequently,1392,Maternity,3,4,Life Sciences,1,...,3,80,0,8,3,3,8,7,3,0
4,1414939,27,No,Travel_Rarely,591,Maternity,2,1,Medical,1,...,4,80,1,6,3,3,2,2,2,2


In [230]:
del df['EmployeeID']

In [231]:
ohe_columns = ['BusinessTravel', 'Department','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']

In [232]:
df.shape

(1676, 34)

In [233]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1676 entries, 0 to 1675
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1676 non-null   int64 
 1   Attrition                 1676 non-null   object
 2   BusinessTravel            1676 non-null   object
 3   DailyRate                 1676 non-null   int64 
 4   Department                1676 non-null   object
 5   DistanceFromHome          1676 non-null   int64 
 6   Education                 1676 non-null   int64 
 7   EducationField            1676 non-null   object
 8   EmployeeCount             1676 non-null   int64 
 9   EnvironmentSatisfaction   1676 non-null   int64 
 10  Gender                    1676 non-null   object
 11  HourlyRate                1676 non-null   int64 
 12  JobInvolvement            1676 non-null   int64 
 13  JobLevel                  1676 non-null   int64 
 14  JobRole                 

In [234]:
[[features,df[features].isnull().sum()] for features in df.columns if df[features].isnull().sum()>0]

[]

In [235]:
df['Attrition'] = np.where(df['Attrition'] == 'Yes', 1, 0)

In [236]:
for cols in ohe_columns:
  df = pd.get_dummies(df[cols])

ValueError: ignored

In [ ]:
df1 = pd.get_dummies(df[ohe_columns])
df1

In [ ]:
df2 = df[df.columns.difference(ohe_columns)]

In [ ]:
pd.concat([df,df2])

In [ ]:
df[ohe_columns]

In [ ]:
X = df.drop(columns =['Attrition'])
y = df['Attrition']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline


In [ ]:
ohe = OneHotEncoder()
ohe.fit(X[ohe_columns])

In [ ]:
column_trans = make_column_transformer(
    (OneHotEncoder(categories=ohe.categories_), ohe_columns)
    ,remainder='passthrough'
    )

In [ ]:
## Create our model
model = Sequential()

# 1st layer: input_dim=8, 12 nodes, RELU
model.add(Dense(63, input_dim=32,  activation='relu'))
# 2nd layer: 8 nodes, RELU
model.add(Dense(32, activation='relu'))
# output layer: dim=1, activation sigmoid
model.add(Dense(1,  activation='sigmoid' ))

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy',   # since we are predicting 0/1
             optimizer='adam',
             metrics=['accuracy'])

In [ ]:
# checkpoint: store the best model
ckpt_model = 'pima-weights.best.hdf5'
checkpoint = ModelCheckpoint(ckpt_model, 
                            monitor='val_acc',
                            verbose=1,
                            save_best_only=True,
                            mode='max')
callbacks_list = [checkpoint]

In [ ]:
pipe = make_pipeline(column_trans, model)
pipe

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)


In [ ]:
# Ensure that fieldnames aren't included
X_train = X_train.values
y_train = y_train.values
X_test  = X_test.values
y_test  = y_test.values

In [ ]:
history = pipe.fit(X_train, y_train, epochs=150, batch_size=10)